In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim import Adam
import torchvision.transforms as transforms

import pandas as pd
import time
import warnings
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f"Using device: {device}")

torch.manual_seed(0)

warnings.filterwarnings('ignore')

Using device: cuda


In [4]:
class FeatureDataset(Dataset):
    def __init__(self, csv_file, transform=None, target_transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        self.target_transform = target_transform
        self.label_map = {
            'DJIM600': 0,
            'DJIS1000': 1,
            'DJIS1000LOUD': 2,
            'MAVIC2PRO': 3,
            'MAVICPRO': 4,
            'MAVICPROPLAT': 5,
            'PHANTOM4': 6,
            'PHANTOM4QP': 7,
        }

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        features = torch.FloatTensor(self.df.iloc[idx, 2])
        label = self.df.iloc[idx, 1]
        
        if self.target_transform:
            label = self.target_transform(label)
        
        return torch.tensor(features), torch.tensor(label)

# Usage example
csv_file = "./AnechoicPeaks.csv"

# Define any transformations if needed
transform = None
target_transform = None

dataset = FeatureDataset(csv_file, transform=transform, target_transform=target_transform)

In [6]:
train_size = int(.95 * len(dataset)) # CHANGE BASED ON SIZE OF DATASET
test_size = int(.75 * (len(dataset) - train_size))
val_size = len(dataset) - train_size - test_size

train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size, val_size])
print(train_dataset)
print(train_dataset[0])

#COUNTS PER CLASS --> ([ 933., 1713.,  405., 1241., 1257., 1233., 1237., 1245.])
label_num = [ 933., 1713.,  405., 1241., 1257., 1233., 1237., 1245.]
label_weights = torch.FloatTensor([1 / num for num in label_num]).to(device)

# Using torch.utils.data.weightedRandomSampler to deal with non-uniform dataset
train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True
    # sampler=WeightedRandomSampler(weights=label_weights, num_samples=train_size, replacement=True)
    )

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False
)

(tensor([ 38.2269, 103.0592, 110.6002,  ...,   0.3175,   0.3175,   0.3175]), tensor(3))


In [10]:
for feature, label in train_loader:
    print(label)

tensor([5, 1, 1, 7, 4, 1, 1, 3, 0, 7, 6, 6, 1, 5, 7, 6, 7, 7, 1, 6, 3, 3, 0, 3,
        6, 6, 6, 1, 4, 4, 4, 2, 1, 4, 3, 3, 5, 4, 5, 7, 5, 5, 0, 3, 4, 5, 6, 6,
        1, 7, 0, 7, 3, 6, 6, 1, 7, 3, 7, 3, 5, 7, 6, 3, 3, 6, 7, 3, 6, 6, 5, 2,
        7, 6, 1, 4, 4, 4, 6, 5, 3, 1, 1, 2, 7, 1, 5, 0, 3, 7, 5, 1, 7, 6, 7, 4,
        2, 5, 4, 6, 6, 7, 6, 1, 5, 3, 5, 3, 6, 6, 3, 6, 7, 6, 1, 7, 6, 4, 5, 0,
        1, 3, 6, 6, 6, 1, 3, 3])


KeyboardInterrupt: 

In [11]:
train_dataset[400][1].shape

torch.Size([])

In [7]:
class LNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            nn.LazyLinear(1024),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(1024),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(512),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(256),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(8)
        )

    def forward(self, x):
        logits = self.seq(x)
        probs = F.softmax(logits, dim=1)
        return probs

In [8]:
def split_seconds(seconds):
    minutes = seconds // 60
    hours = minutes // 60
    days = hours // 24
    return seconds % 60, minutes % 60, hours % 24, days

In [15]:
from tqdm.notebook import tqdm

def main():
    # Define model
    model = LNN()
    # Cuda setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")
    model = model.to(device)
   
    print(f"Using device: {device}")
    # Optimizer setup
    optimizer = Adam(model.parameters(), lr=1e-4)
   
    # Loss function
    #COUNTS PER CLASS --> ([ 933., 1713.,  405., 1241., 1257., 1233., 1237., 1245.])
    label_num = [ 933., 1713.,  405., 1241., 1257., 1233., 1237., 1245.]
    label_weights = torch.FloatTensor([1 / num for num in label_num]).to(device)
    loss_fn = nn.CrossEntropyLoss(reduction="mean")
   
    # Number of epochs
    num_epochs = 64
   
    # Train or load model?
    print("Training model....")
    start = time.time()
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
       
        model.train()
        for batch_idx, (features, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)):
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
           
            # CNN forward pass
            probabilities = model(features)
            loss = loss_fn(probabilities, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        num_test = 0
        num_correct = 0
        with torch.no_grad():
            for features, labels in tqdm(test_loader, desc="Testing"):
                features, labels = features.to(device), labels.to(device)
                probabilities = model(features)
                _, preds = probabilities.max(1)
                num_test += labels.size(0)
                num_correct += preds.eq(labels).sum().item()
    
        tqdm.write(f"Epoch {epoch+1}/{num_epochs}, Test accuracy: {num_correct / num_test * 100:.2f}")

        torch.save(model.state_dict(), f"./peaks_models/_epoch_{str(epoch)}.pt")
   
    end = time.time()
    seconds, minutes, hours, days = split_seconds(end - start)
    print(f"Training Runtime: {int(days)}d {int(hours)}h {int(minutes)}m {seconds:.2f}s")
   
    # Evaluate model on validation data
    model.eval()
    print("Evaluating model....")
    start = time.time()
    num_test = 0
    num_correct = 0
   
    with torch.no_grad():
        for features, labels in tqdm(val_loader, desc="Validating"):
            features, labels = features.to(device), labels.to(device)
            probabilities = model(features)
            _, preds = probabilities.max(1)
            num_test += labels.size(0)
            num_correct += preds.eq(labels).sum().item()
   
    print(f"Test accuracy: {num_correct / num_test * 100:.2f}%")
    end = time.time()
    seconds, minutes, hours, days = split_seconds(end - start)
    print(f"Testing Runtime: {int(days)}d {int(hours)}h {int(minutes)}m {seconds:.2f}s")

In [16]:
main()

Using device: cuda
Training model....


Epochs:   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 1/64:   0%|          | 0/69 [00:00<?, ?it/s]

In [49]:
# Define model
model = LNN()
# Cuda setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model = model.to(device)

print(f"Using device: {device}")

# Optimizer setup
optimizer = Adam(model.parameters(), lr=1e-3)


state = torch.load(r"C:\Users\Alec\Desktop\afrl-uav-detection\classify\1719597586.3545992_epoch_0.pt")
model.load_state_dict(state)


model.eval()
print("Evaluating model....")
start = time.time()
num_test = 0
num_correct = 0

with torch.no_grad():
    with Progress(
        TextColumn("[progress.description]{task.description}"),
        BarColumn(),
        MofNCompleteColumn(),
        TimeRemainingColumn(),
        refresh_per_second=10
    ) as progress:
        test_task = progress.add_task("[yellow]Testing", total=len(test_loader))
        for audio_file, images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            probabilities = model(images)
            _, preds = probabilities.max(1)
            num_test += labels.size(0)
            num_correct += preds.eq(labels).sum().item()
            progress.update(test_task, advance=1)
            progress.refresh()

print(f"Test accuracy: {num_correct / num_test * 100:.2f}%")
end = time.time()
seconds, minutes, hours, days = split_seconds(end - start)
print(f"Testing Runtime: {int(days)}d {int(hours)}h {int(minutes)}m {seconds:.2f}s")

Using device: cuda


Output()

Evaluating model....


Test accuracy: 25.40%
Testing Runtime: 0d 0h 0m 2.33s
